# Collect Data

The first step in building a price prediction model is to collect data.


French governement provides real estate sales data on its open data platform: <https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/><br />
A file is provided for each year, since 2016, and includes all real estate sales data for France.<br />
In consequence, each file is large: more than 400 Mo per file for last years; a total of around 2 Go of data for the last 6 years.<br />
In our case, we are interested only by sales done in Paris for our prediction model, so only a part of each file is required.


<b>Notebook objective:</b> create a unique dataset from all the files provided, but keeping only data from Paris.

Note: 
* In this notebook, we focus only in filtering data based on the sales location.
* We will focus on data analysis for building a prediction model in a separated and dedicated notebook => [02_BuildModel](02_BuildModel.ipynb)

## Data Analysis

Analyze dataset structure and content to find a way to get only sales data from Paris.

In [98]:
import pandas as pd

df_data = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/817204ac-2202-4b4a-98e7-4184d154d98c', sep='|', low_memory=False)
print(f'DataFrame Shape: {df_data.shape}')

DataFrame Shape: (1210569, 43)


The dataset provides 43 columns.

Check the columns to find possible olumns to filter data on sales location:

In [99]:
print(df_data.columns)

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


The following columns are related to sales location:
* 'Code Postal' (postal code)
* 'Commune' (city name)
* 'Code departement' (french subarea)

We analyze each column to check if we can filter on:

In [100]:
location_cols = ['Code departement', 'Code postal', 'Commune']
for col in location_cols:
    print(f'{col}: {df_data[col].dtype}')
    print(df_data[col].unique())
    print('---------------')

Code departement: object
['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '58' '59' '60' '61' '62' '63' '64' '65' '66' '69' '70' '71' '72'
 '73' '74' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86' '87'
 '88' '89' '90' '91' '92' '93' '94' '95' '971' '972' '973' '974' '75']
---------------
Code postal: float64
[ 1370.  1290.  1310. ... 75013. 75012. 75011.]
---------------
Commune: object
['VAL-REVERMONT' 'BEY' 'BUELLAS' ... 'PARIS 15' 'PARIS 16' 'PARIS 11']
---------------


Report :
* 'Code departement': is a french sub area identifier. Paris is identied with the code 75 (the code used as the 2 first characters of postal code)
* 'Code postal': Described as a float, it is in fact a in integer that starts with the 'code departement' => french nomenclature
* 'Commune': is the city name => We can filter on the name begining by 'PARIS'

### Code departement (french sub area)

In [101]:
df_data_dep = df_data.loc[df_data["Code departement"] == '75']
print(f'Code departement = 75: {df_data_dep.shape[0]} rows')
print(df_data_dep['Commune'].value_counts())

Code departement = 75: 33567 rows
PARIS 16    3448
PARIS 15    3385
PARIS 18    3046
PARIS 17    2986
PARIS 14    2514
PARIS 11    2389
PARIS 19    2126
PARIS 20    1993
PARIS 12    1616
PARIS 10    1576
PARIS 13    1544
PARIS 09    1312
PARIS 07    1208
PARIS 08    1084
PARIS 06     764
PARIS 05     694
PARIS 03     644
PARIS 04     469
PARIS 02     458
PARIS 01     311
Name: Commune, dtype: int64


### Code Postal (Postal code)

By default, a postal code starts with the department code. \
So, at first try we filter based on 'Code postal' column beggining with '75':

In [102]:
df_data_cp = df_data.loc[df_data['Code postal'].astype(str).str.startswith('75')]
print(f'Code postal - Start with "75": {df_data_cp.shape[0]} rows')
print(df_data_cp['Code postal'].value_counts())

Code postal - Start with "75": 33789 rows
75016.0    3460
75015.0    3385
75018.0    3045
75017.0    2982
75014.0    2513
75011.0    2389
75019.0    2123
75020.0    1991
75012.0    1611
75010.0    1576
75013.0    1464
75009.0    1312
75007.0    1140
75008.0    1084
75006.0     764
75005.0     694
75003.0     644
75004.0     468
75002.0     458
75001.0     311
7560.0      111
7590.0      101
7510.0      101
7530.0       34
7580.0       28
Name: Code postal, dtype: int64


The previous code returns postal codes of Paris (75001 to 75020), but also postal codes of other cities :

In [103]:
df_cities = df_data_cp.loc[df_data_cp['Code postal'].astype(str).isin(['7560.0', '7590.0', '7510.0', '7530.0', '7580.0'])]
print(df_cities['Commune'].value_counts())

MONTPEZAT-SOUS-BAUZON             76
SAINT-ETIENNE-DE-LUGDARES         48
LE ROUX                           35
SAINT-LAURENT-LES-BAINS-LAVAL-    33
CROS-DE-GEORAND                   29
SAINTE-EULALIE                    29
MAZAN-L'ABBAYE                    27
SAINT-JEAN-LE-CENTENIER           26
SAINT-CIRGUES-EN-MONTAGNE         15
GENESTELLE                        13
CELLIER-DU-LUC                     8
MEZILHAC                           7
VALLEES-D'ANTRAIGUES-ASPERJOC      6
SAINT-ALBAN-EN-MONTAGNE            6
LE PLAGNAL                         5
AIZAC                              4
LACHAMP-RAPHAEL                    3
BERZEME                            2
BORNE                              1
LAVIOLLE                           1
USCLADES-ET-RIEUTORD               1
Name: Commune, dtype: int64


For the provided codes, the cities are not Paris.

In fact, postal codes are, in France, normally defined on 5 characters : 2 characters for the department code + 3 characters specific to the city in this departement.<br />
So, in the original file, when a postal code is by example 7560 we should consider it as 07560 => department code 07.

To get only sales for Paris postal code we have to filter on column 'Code postal' where:
* length is 5 characters
* and it starts with 75

In [104]:
is_cp_paris = df_data['Code postal'].astype(str).map(lambda cp: (cp != 'nan') & (len(cp.replace('.0', '')) == 5) & (cp.startswith('75')))
df_data_cp = df_data.loc[is_cp_paris]
print(f'Code postal - Start with "75": {df_data_cp.shape[0]} rows')
print(df_data_cp['Code postal'].value_counts())

Code postal - Start with "75": 33414 rows
75016.0    3460
75015.0    3385
75018.0    3045
75017.0    2982
75014.0    2513
75011.0    2389
75019.0    2123
75020.0    1991
75012.0    1611
75010.0    1576
75013.0    1464
75009.0    1312
75007.0    1140
75008.0    1084
75006.0     764
75005.0     694
75003.0     644
75004.0     468
75002.0     458
75001.0     311
Name: Code postal, dtype: int64


Now we only have rows corresponding to one of the postal codes associated to Paris.

### Commune (City name)

In [105]:
df_data_com = df_data.loc[df_data.Commune.str.startswith('PARIS')]
print(f'Commune - Start with "Paris": {df_data_com.shape[0]} rows')
print(df_data_com['Commune'].value_counts())

Commune - Start with "Paris": 33643 rows
PARIS 16           3448
PARIS 15           3385
PARIS 18           3046
PARIS 17           2986
PARIS 14           2514
PARIS 11           2389
PARIS 19           2126
PARIS 20           1993
PARIS 12           1616
PARIS 10           1576
PARIS 13           1544
PARIS 09           1312
PARIS 07           1208
PARIS 08           1084
PARIS 06            764
PARIS 05            694
PARIS 03            644
PARIS 04            469
PARIS 02            458
PARIS 01            311
PARISOT              47
PARIS-L HOPITAL      29
Name: Commune, dtype: int64


We cannot filter only with city name starting with PARIS, because other cities have the same root string.

But we see that for Paris, the city name is 'PARIS' + the 'arrondissement' code (Paris sub division), between 1 and 20.<br />
So we can filter using a dedicated list of city names corresponding only to Paris :

In [106]:
paris_arr = [f'PARIS {arr:02d}' for arr in range(1, 21)]

df_data_com = df_data.loc[df_data.Commune.isin(paris_arr)]
print(f'Commune - Start with "Paris": {df_data_com.shape[0]} rows')
print(df_data_com['Commune'].value_counts())

Commune - Start with "Paris": 33567 rows
PARIS 16    3448
PARIS 15    3385
PARIS 18    3046
PARIS 17    2986
PARIS 14    2514
PARIS 11    2389
PARIS 19    2126
PARIS 20    1993
PARIS 12    1616
PARIS 10    1576
PARIS 13    1544
PARIS 09    1312
PARIS 07    1208
PARIS 08    1084
PARIS 06     764
PARIS 05     694
PARIS 03     644
PARIS 04     469
PARIS 02     458
PARIS 01     311
Name: Commune, dtype: int64


## Generate sample dataset
In this section we generate a unique CSV file concataining all files provided by open data platform, and filtering to keep only Paris data.

We use the 3 filters previously defined, combining them with 'OR' operator to keep rows where a column criteria is:
* not filled in
* not aligned to other columns criteria (by example postal code is correct for Paris, but not the department code)

These cases will be processed during data analysis in 'build model' step of this project.

In [107]:
import os.path

def build_sample(files: dict, output_path: str):
    print('-------------------')
    print('Build sample: Start')
    print('-------------------')
    if os.path.exists(output_path):
        os.remove(output_path)

    paris_arr = [f'PARIS {arrondissement:02d}' for arrondissement in range(1, 21)]
    index = 0
    for year, file in files.items():
        print(f'--- Process year: {year}')

        # load file
        df = pd.read_csv(file, sep='|', low_memory=False)
        print(f'\tImport - Shape: {df.shape}')

        # filter to get sample
        paris_department = df['Code departement'] == 75
        paris_postalcode = df_data['Code postal'].astype(str).map(lambda cp: (cp != 'nan') & (len(cp.replace('.0', '')) == 5) & (cp.startswith('75')))
        paris_cityname = df.Commune.isin(paris_arr)
        df_paris = df.loc[paris_department | paris_postalcode | paris_cityname]
        print(f'\tSample - Shape: {df_paris.shape}')        

        # append to csv file
        df_paris.to_csv(output_path, mode='a', header=(index>0))
        index += 1
        print(f'\tSample for {year} is {round(100* df_paris.shape[0]/df.shape[0], 2)}% of full dataset for {year}')
        del df, df_paris

    print('------------------')
    print('Build sample: Done')
    print('------------------')

files = {
    '2021':'https://www.data.gouv.fr/fr/datasets/r/817204ac-2202-4b4a-98e7-4184d154d98c',
    '2020':'https://www.data.gouv.fr/fr/datasets/r/90a98de0-f562-4328-aa16-fe0dd1dca60f',
    '2019':'https://www.data.gouv.fr/fr/datasets/r/3004168d-bec4-44d9-a781-ef16f41856a2',
    '2018':'https://www.data.gouv.fr/fr/datasets/r/1be77ca5-dc1b-4e50-af2b-0240147e0346',
    '2017':'https://www.data.gouv.fr/fr/datasets/r/7161c9f2-3d91-4caf-afa2-cfe535807f04'
}

build_sample(files, './data/raw/real_estate_sales.csv')

-------------------
Build sample: Start
-------------------
--- Process year: 2021
	Import - Shape: (1210569, 43)
	Sample - Shape: (33586, 43)
	Sample for 2021 is 2.77% of full dataset for 2021
--- Process year: 2020
	Import - Shape: (3149482, 43)
	Sample - Shape: (87910, 43)
	Sample for 2020 is 2.79% of full dataset for 2020
--- Process year: 2019
	Import - Shape: (3612746, 43)
	Sample - Shape: (103561, 43)
	Sample for 2019 is 2.87% of full dataset for 2019
--- Process year: 2018
	Import - Shape: (3332865, 43)
	Sample - Shape: (101590, 43)
	Sample for 2018 is 3.05% of full dataset for 2018
--- Process year: 2017
	Import - Shape: (3383409, 43)
	Sample - Shape: (98335, 43)
	Sample for 2017 is 2.91% of full dataset for 2017
------------------
Build sample: Done
------------------
